In [ ]:
# Instalando bibliotecas 
!pip install --upgrade tweepy
!pip install snscrape
!pip install --upgrade networkx

In [ ]:
# Importando biblioteca de acesso a API do Twitter
import tweepy as tw
#tw.__version__   aaa sb
import pytz
import datetime
import pandas as pd
import networkx as nx
pd.set_option('display.max_colwidth', None)

In [ ]:
class GrafoTwitter:
  dados = []
  usuarios = []
  grafo = None

  #Construtor
  def __init__(self, query, data_inicio, data_fim,bearer_token,client=None):
    self.query = query
    self.data_inicio = data_inicio
    self.data_fim = data_fim
    self.bearer_token = bearer_token
    self.client = None,
    self.grafo = nx.DiGraph()
  
  # Autenticar usando chave e retorna objeto da API para manipulacao
  def autenticarAPI(self):
    self.client = tw.Client(self.bearer_token)
    print(self.client)
    

  # Formatar a data de acordo com o necessário na API
  def converteData(self,data):
    # O formato de datetime esperado pela API é "2023-1-1 12:00:00"
    local_time = pytz.timezone("America/Sao_Paulo")
    utc_data = local_time.localize(datetime.datetime.strptime(data, "%Y-%m-%d %H:%M:%S"), is_dst=None).astimezone(pytz.utc)
    dtformat = '%Y-%m-%dT%H:%M:%SZ'
    time = datetime.datetime.utcnow()
    return utc_data.strftime(dtformat)
  
  # Extrair 100 tweets (limite da API)
  def extrairTweets(self):
    self.tweetsContent = self.client.search_recent_tweets(self.query, max_results=100,start_time= self.converteData(self.data_inicio), end_time= self.converteData(self.data_fim),
                                       user_fields=['id','username'], expansions=['author_id','entities.mentions.username'], tweet_fields=['created_at'])
    self.df_tweets_extraidos = pd.DataFrame.from_dict(self.tweetsContent.data)
    print(self.df_tweets_extraidos.head(3))

  # Contar Tweets disponiveis na query pesquisada. 
  def contarTweetsQuery(self):
    #Importante pois a cada 15 minutos há um limite de 450 request, ou seja, 45000 tweets podem ser extraidos a cada 15 minutos.
    contagem = self.client.get_recent_tweets_count(self.query, start_time=self.converteData(self.data_inicio), end_time=self.converteData(self.data_fim), granularity="day")
    #print('Quantidade de Tweets:' + str(contagem.meta['total_tweet_count']))
    return contagem.meta['total_tweet_count']

  def gravarDicionario(self):
    for user in self.tweetsContent.includes['users']:
      users_dict = {}
      users_dict[0] = user['id']
      users_dict[1] = user['username']
      self.usuarios.append(users_dict)

  def criarGrafo(self):
    self.gravarDicionario()
    for tweet in self.tweetsContent.data:
        self.dados.append([tweet.author_id, tweet.created_at, tweet.edit_history_tweet_ids, tweet.id, tweet.text])
        if "entities" in tweet:
          for mencao in tweet.entities['mentions']:            
              # Buscando o user_name do autor do Tweet
              username_autor_tweet = list(filter(lambda usuario: usuario[0] == tweet.author_id, self.usuarios))[0][1]
              # Criando uma aresta do autor Tweet para o(s) usuário(s) citado(s)
              self.grafo.add_edge(username_autor_tweet,mencao['username'])
    # Exportando o grafo para um arquivo            
    
    if ("next_token" in self.tweetsContent.meta):
      self.fazerPaginacao()
    nx.write_graphml_lxml(self.grafo, "grafo_extracao_{0}.graphml".format(self.query.split(' ',1)[0])) 

  def fazerPaginacao(self):
    while ("next_token" in self.tweetsContent.meta):
       self.tweetsContent = self.client.search_recent_tweets(self.query, next_token=self.tweetsContent.meta['next_token'], max_results=100,
                                           start_time=self.converteData(self.data_inicio), end_time= self.converteData(self.data_fim),user_fields=['id','username'], 
                                           expansions=['author_id','entities.mentions.username'], tweet_fields=['created_at'])
    
    self.gravarDicionario()
    # Gravando a ID e username dos autores dos Tweets
    for user in self.tweetsContent.includes['users']:
        users_dict = {}
        users_dict[0] = user['id']
        users_dict[1] = user['username']
        self.usuarios.append(users_dict)
    
    for tweet in self.tweetsContent.data:
        self.dados.append([tweet.author_id, tweet.created_at, tweet.edit_history_tweet_ids, tweet.id, tweet.text])
        if "entities" in tweet:
          for mencao in tweet.entities['mentions']:            
              # Gravando a ID e username dos autores dos Tweets
              username_autor_tweet = list(filter(lambda usuario: usuario[0] == tweet.author_id, self.usuarios))[0][1]
              # Criando uma aresta do autor Tweet para o(s) usuário(s) citado(s)
              self.grafo.add_edge(username_autor_tweet,mencao['username'])


    

In [ ]:
# Main Script para executar o grafo para o Lula
if __name__ == "__main__":
  grafinho = GrafoTwitter(query='Lula  lang:pt', data_inicio='2023-01-08 23:05:00', data_fim='2023-01-08 23:10:00',bearer_token='AAAAAAAAAAAAAAAAAAAAAKtckwEAAAAAYV8r7pZHTOzX%2F1ZH7dywUUELKxU%3DfuB1CZ6BYXp8S0Jjf2md95jIifYwCLrc875I0Zt9kp5cNOsd6S')
  grafinho.autenticarAPI()
  grafinho.extrairTweets()
  grafinho.contarTweetsQuery()
  grafinho.criarGrafo()

In [ ]:
# Main Script para executar o grafo para o Bolsonaro
if __name__ == "__main__":    
  grafinho = GrafoTwitter(query='Bolsonaro  lang:pt', data_inicio='2023-01-08 23:05:00', data_fim='2023-01-08 23:10:00',bearer_token='AAAAAAAAAAAAAAAAAAAAAKtckwEAAAAAYV8r7pZHTOzX%2F1ZH7dywUUELKxU%3DfuB1CZ6BYXp8S0Jjf2md95jIifYwCLrc875I0Zt9kp5cNOsd6S')
  grafinho.autenticarAPI()
  grafinho.extrairTweets()
  grafinho.contarTweetsQuery()
  grafinho.criarGrafo()